In [1]:
# classifier: cats vs. dogs
# dataset: https://www.kaggle.com/c/dogs-vs-cats

# ---------------------
# import required packages
# ---------------------
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from random import shuffle
from tqdm import tqdm
import torchvision
from sklearn.model_selection import train_test_split


In [2]:
# ---------------------
# define constants
# ---------------------
TRAIN_DIR = '/home/rabina7/Downloads/cat-and-dog/train_set'
TEST_DIR = '/home/rabina7/Downloads/cat-and-dog/test_set'

IMG_SIZE = 50 # resize all images to the same size
LR = 1e-3 # learning rate: 0.001

# model name, for model loading and saving
MODEL_NAME = 'catsvsdogs-{}-{}.model'.format(LR, '6conv-basic')



In [3]:
# output of our model is an array consists 2 elements: [catniss, dogniss]
# each element represents the probability of being each animal.
# e.g., [1, 0] means this image is 100% a cat image and 0% a dog image.
# So, the final label of the image is cat.

categories = ['cat', 'dog']

def img_label(img):
    # get the true label of the image
    # images in training set are named in form of <animal>.xx.jpg, xx are numbers.
    # print("img path: "+img)
    #parts = img.split('.')
    #print("length of parts: "+str(len(parts)))
    #print("parts: "+str(parts))
    
    text_label = img.split('.')[-3]
    
    if text_label == categories[0]:
        return [1, 0]
    elif text_label == categories[1]:
        return [0, 1]
    else:
        raise ValueError('The image does not belong to any category! ' + text_label)

In [4]:
# ---------------------
# process training data
# ---------------------
def process_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        # full path of the image
        img_full_path = os.path.join(TRAIN_DIR, img)
        # true label of the image
        true_label = img_label(img)
        # load the image
        img = cv2.imread(img_full_path, cv2.IMREAD_GRAYSCALE)
        # resize the image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        # add new training data, in form of
        # (img, label)
        training_data.append([np.array(img), np.array(true_label)])
        #training_data.append([np.array(img), img_idx])
    # shuffle the dataset
    shuffle(training_data)
    # save the dataset so that next time we can load it directly
    np.save('trainset-{}.npy'.format(IMG_SIZE), training_data)
    
    return training_data       

In [5]:
# ---------------------
# process testing data
# ---------------------
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        # full path of the image
        img_full_path = os.path.join(TEST_DIR, img)
        #img_idx = img.split('.')[0]
        
        true_label = img_label(img)
         # load the image
        img = cv2.imread(img_full_path, cv2.IMREAD_GRAYSCALE)
        # resize the image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        # add new testing data, in form of
        # (img, img_idx)
        testing_data.append([np.array(img), np.array(true_label)])
        #calculating the testing
        #simple loop that goes through all the 
        #API to predict the label and compare to the true label
        #counter
    shuffle(testing_data)
    # save the dataset so that next time we can load it directly
    np.save('testset-{}.npy'.format(IMG_SIZE), testing_data)
    return testing_data

In [6]:
# ---------------------
# load training data
# ---------------------
def load_train_data(load_exist):
    trainset = None
    if load_exist and os.path.exists('trainset-{}.npy'.format(IMG_SIZE)):
        # if want to load an existing training set
        trainset = np.load('trainset-{}.npy'.format(IMG_SIZE), allow_pickle = True)
    else:
        trainset = process_train_data()
    return trainset

In [7]:
# ---------------------
# construct the cnn model for this project
# ---------------------
import tflearn

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

def conv_net():
    convnet = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1],
                        name = 'input')
    # conv_2d(incoming, nb_filter, filter_size, ..., activation)
    convnet = conv_2d(convnet, 32, 5, activation = 'relu')
    # max_pool_2d(incoming, kernel_size, ...)
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 64, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 32, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 64, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 32, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 64, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
      
    # fully_connected(incoming, n_units, activation, ...)
    #convnet = fully_connected(convnet, 1024, activation = 'relu')
    # dropout(incoming, keep_prob)
    #convnet = dropout(convnet, 0.8)
    
    #standard recommendation for the Net arch    
    convnet = fully_connected(convnet, 2, activation = 'softmax')
    
    # regression(incoming, optimizer, learning_rate, loss, name, ...)
    convnet = regression(convnet, optimizer = 'adam', learning_rate = LR,
                             loss = 'categorical_crossentropy', name = 'targets')

    return convnet

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# ---------------------
# define the model
# ---------------------
convnet = conv_net()
model = tflearn.DNN(convnet, tensorboard_dir = 'log')


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [9]:
# ---------------------
# train the model
# ---------------------

# load model if exists
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print ('loaded existing model', MODEL_NAME)
else:
    # Loading the training data-set after pre-processing
    # and splitting the testing data and validation data

    train_data = load_train_data(False)
    train = train_data[:-500]
    validation = train_data[-500:]

    # Setting up the features and lables
    # x-Features & y-Labels
    train_x = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    train_y = [i[1] for i in train]
    validation_x = np.array([i[0] for i in validation]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    validation_y = [i[1] for i in validation]
    # Training the CNN model with the data-sets
    model.fit({'input': train_x}, {'targets': train_y}, n_epoch = 15,
                        validation_set = ({'input': validation_x}, {'targets': validation_y}),
                        snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)

    # save the trained model
    #model.save(MODEL_NAME)

Training Step: 1769  | total loss: 0.13944 | time: 12.203s
| Adam | epoch: 015 | loss: 0.13944 - acc: 0.9424 -- iter: 7488/7505
Training Step: 1770  | total loss: 0.13401 | time: 13.338s
| Adam | epoch: 015 | loss: 0.13401 - acc: 0.9450 | val_loss: 0.91638 - val_acc: 0.7260 -- iter: 7505/7505
--


In [10]:
# the model acheives 90% accuracy, save it if not saved yet
if not os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.save(MODEL_NAME)

INFO:tensorflow:/home/rabina7/Downloads/catsvsdogs-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
# ---------------------
# load testing data
# ---------------------
def load_test_data(load_exist):
    testset = None
    if load_exist and os.path.exists('testset-{}.npy'.format(IMG_SIZE)):
        # if want to load an existing testing set
        testset = np.load('testset-{}.npy'.format(IMG_SIZE), allow_pickle = True)
    else:
        testset = process_test_data()
    return testset

In [12]:
test_data = load_test_data(False)
test_x = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test_data] 

predictions = model.predict(test_x)
total_test_samples = len(test_data)

counter = 0
for prediction, true_label in zip(predictions, test_y):
    predicted_class = np.argmax(prediction) # [0.6, 0.4] #predicts whichever is max
    #print("true_lable:"+str(true_label))
    actual_class = np.argmax(true_label) # [0, 1], [1, 0]
    if(predicted_class == actual_class):
        counter+=1
        
    print("predicate class: {}, True label: {}, counter {} ".format(predicted_class, actual_class, counter))

# For integers, ‘/’ operator returns the quotient. For example, 3/10 returns 0.

# So, add 1.0 before counter to convert it to a float number. 

accuracy = 1.0 * counter / total_test_samples

print("The testing accuracy is {}.".format(accuracy))


100%|██████████| 1641/1641 [00:01<00:00, 1201.95it/s]


predicate class: 0, True label: 1, counter 0 
predicate class: 1, True label: 1, counter 1 
predicate class: 0, True label: 0, counter 2 
predicate class: 1, True label: 0, counter 2 
predicate class: 0, True label: 0, counter 3 
predicate class: 1, True label: 1, counter 4 
predicate class: 1, True label: 1, counter 5 
predicate class: 1, True label: 0, counter 5 
predicate class: 0, True label: 0, counter 6 
predicate class: 1, True label: 1, counter 7 
predicate class: 1, True label: 1, counter 8 
predicate class: 0, True label: 0, counter 9 
predicate class: 1, True label: 0, counter 9 
predicate class: 0, True label: 0, counter 10 
predicate class: 0, True label: 1, counter 10 
predicate class: 1, True label: 1, counter 11 
predicate class: 0, True label: 0, counter 12 
predicate class: 0, True label: 1, counter 12 
predicate class: 0, True label: 0, counter 13 
predicate class: 1, True label: 1, counter 14 
predicate class: 1, True label: 1, counter 15 
predicate class: 0, True l

In [13]:
clip_min = 0
clip_max = 1
# perturbation -- FGSM
def fgsm(image, epsilon, data_grad):
    '''
    F(ast) G(radient) S(ign) M(ethod)
    by Ian J. Goodfellow, Jonathon Shlens & Christian Szegedy 
    Explaining and harnessing adversarial examples
    ICLR 2015
    '''
    # generate the perturbation for given image,
    # by adjusting each pixel of the image.
    perturbed_image = image + epsilon * data_grad.sign()
    perturbed_image = torch.clamp(perturbed_image, clip_min, clip_max)
    return perturbed_image

In [14]:
# generate adversarial examples
        
def generate(model, device, load_test_data, epsilon):
    '''
    generate adversarial example over images in test_loader.
    '''
    
    #first = True # for debugging
    # accuracy counter
    correct = 0
    # track example
    adversarial_examples = []
    
    # iterate examples in testset
    for image, target in load_test_data:
        # send image and label to the device
        original_image = image.numpy().squeeze()
        
        if first:
            # for debuggig
            plt.imshow(original_image)
            first = False
        image = image.to(device)
        target = target.to(device)
        
        # set requires_grad attribute of tensor
        # for gradient is needed for generating perturbation
        image.requires_grad = True
        
        # forward propagation
        img = image.view(image.shape[0], -1)
        original_output = model.forward(img)
        original_pred = original_output.max(1, keepdim = True)[1]
        
        if target.item() != original_pred.item():
#             If the image was incorrectly classified before perturbed,
#             we can simply skip it. It does not make sense to generate
#             adversarial example for such image.
            continue
        
        # compute the loss
        loss = F.nll_loss(original_output, target)
        
        # zero all existing gradients
        model.zero_grad()
        # compute gradients of model in backprop pass
        loss.backward()
        
        # collect data gradient
        data_grad = image.grad.data
        # adversarial attack by FGSM
        perturbed_image = fgsm(image, epsilon, data_grad)
        
        # classify perturbed image
        perturbed_img = perturbed_image.view(perturbed_image.shape[0], -1)
        perturbed_output = model.forward(perturbed_img)
        perturbed_pred = perturbed_output.max(1, keepdim = True)[1]
        
        if target.item() == perturbed_pred.item():
            correct += 1
            # special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adversarial_examples) < 5):
                adv_ex = perturbed_image.squeeze().detach().cpu().numpy()
                adversarial_examples.append((original_pred.item(),
                                            perturbed_pred.item(),
                                            original_image,
                                            adv_ex))
        else:
            if (len(adversarial_examples) < 5):
                adv_ex = perturbed_image.squeeze().detach().cpu().numpy()
                adversarial_examples.append((original_pred.item(),
                                            perturbed_pred.item(),
                                            original_image,
                                            adv_ex))
    
    # compute accuracy for current epsilon
    accuracy = correct / float(len(load_test_data))
    print ('epsilon: {}\ttest accuracy = {} / {} = {}'.format(epsilon, correct, len(laod_test_data), accuracy))

    return accuracy, adversarial_examples


In [15]:
epsilons = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

test_data = load_test_data(True)

# attacking process
accuracies = []
examples = []

# attack for each epsilon
for eps in epsilons:
    acc, ex = generate(model, device, test_data, eps)
    accuracies.append(acc)
    examples.append(ex)

NameError: name 'device' is not defined

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step = 0.1))
plt.xticks(np.arange(0, .6, step = 0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("epsilon")
plt.ylabel("accuracy")
plt.show()

In [ ]:
# -------------------
# use trained model to predict given image
# -------------------
import matplotlib.pyplot as plt

test_data = load_test_data(True)
fig = plt.figure(figsize = (10, 8))

# test and plot 12 images
for i, data in enumerate(test_data[:12]):
    # cat: [1, 0]; dog: [0, 1]
    img_data = data[0]
    img_idx = data[1]
    
    sub_fig = fig.add_subplot(3, 4, i + 1)
    
    orig_data = img_data
    # flatten image
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    
    output = model.predict([data])[0]
    
    if np.argmax(output) == 1:
        pred_label = 'dog'
    else:
        pred_label = 'cat'
    
    sub_fig.imshow(orig_data, cmap = 'gray')
    
    # predicted label and confidence
    plt.title(pred_label + '; ' + str(np.max(output)))
    sub_fig.get_xaxis().set_visible(False)
    sub_fig.get_yaxis().set_visible(False)
    
plt.show()